# M1. Actividad 1
## Robot de limpieza reactivo

### Equipo 2:
- Carlos Tadeo Pérez Capistrán, A01197315
- David Fernando Armendáriz Torres, A01570813
- Nicolás Herrera Hernandez, A01114972




## Imports
 

In [1]:
# 'Model' sirve para definir los atributos a nivel del modelo, maneja los agentes
# 'Agent' es la unidad atómica y puede ser contenido en múltiples instancias en los modelos
from mesa import Agent, Model 

# 'SingleGrid' sirve para forzar a un solo objeto por celda (nuestro objetivo en este "juego")
from mesa.space import SingleGrid, MultiGrid

# 'SimultaneousActivation' habilita la opción de activar todos los agentes de manera simultanea.
from mesa.time import SimultaneousActivation

# 'DataCollector' permite obtener el grid completo a cada paso (o generación), útil para visualizar
from mesa.datacollection import DataCollector

from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

# 'matplotlib' lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

# Definimos los siguientes paquetes para manejar valores númericos: 'numpy' & 'pandas'
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

## Crear el modelo y los agentes


In [2]:
# Agente de la aspiradora o robot limpiador
class RobotLimpiador(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    # Se inicializan los movimientos y el valor de suciedad, este nos va a servir para graficar.
    self.dirty = 99
    self.moves = 0

# Función del agente para mover a la aspiradora a una celda vecina
  def move(self):
    possible_steps = self.model.grid.get_neighborhood(
      self.pos,
      moore=True,
      include_center=False)
    new_position = self.random.choice(possible_steps)
    self.model.grid.move_agent(self, new_position)
    self.moves += 1

# Función del agente para limpiar la celda en la que se encuentra en caso de que este sucia
# y reducir el numero de celdas sucias en el modelo
  def clean(self):
     for cell in self.model.grid.coord_iter():
            cell_content, x, y = cell
            for content in cell_content:
                if isinstance(content, Baldosa):
                  if content.pos == self.pos:
                    if content.dirty == 1:
                      content.dirty = 0
                      self.model.clean -= 1
  
  def step(self):
    self.clean()
    self.move()

# Agente de la baldosa
class Baldosa(Agent):
  def __init__(self, unique_id, model, status):
    super().__init__(unique_id, model)
    # Se inicializan los movimientos y el valor de suciedad, donde 1 = sucio y 0 = limpio.
    self.dirty = status
    self.moves = 0
    


class Modelo(Model):
  def __init__(self, N, width, height, D):
    self.num_agents = N
    self.grid = MultiGrid(width, height, True)
    self.schedule = SimultaneousActivation(self)
    self.running = True
    self.dirty_amount = round((width * height) * D)
    self.clean = self.dirty_amount
    self.dirty_list = [[1,1]]
    
    # Ubicamos a cada las aspiradoras en la grilla de acuerdo al numero de agentes indicado
    for i in range(self.num_agents):
      a = RobotLimpiador(i, self)
      self.schedule.add(a)
      self.grid.place_agent(a, (1, 1))

    b = Baldosa((1, 1), self, 0)
    self.schedule.add(b)
    self.grid.place_agent(b, (1,1))
    
    # Ubicamos a cada baldosa sucia en la grilla
    cont = 0
    while cont != self.dirty_amount:
      x = self.random.randrange(self.grid.width)
      y = self.random.randrange(self.grid.height)
      b = Baldosa((x, y), self, 1)
      if ([x,y] not in self.dirty_list):
        self.schedule.add(b)
        self.grid.place_agent(b, (x,y))
        cont = cont + 1
      self.dirty_list.append([x,y])  
    
    # Ubicamos a cada baldosa limpia en la grilla
    for (content, x, y) in self.grid.coord_iter():
      if ([x,y] not in self.dirty_list):
        b = Baldosa((x, y), self, 0)
        self.schedule.add(b)
        self.grid.place_agent(b, (x, y))

    # Aquí definimos el colector de datos para obtener el grid completo y los parametros de los agentes.
    self.datacollector = DataCollector(
    model_reporters={"Grid": self.get_grid}, agent_reporters={"Dirty": "dirty", "Moves": "moves"}
    )

  def step(self):
    self.schedule.step()
    self.datacollector.collect(self)


  def get_grid(self):

      # Generamos la grid para contener los valores
      grid = np.zeros((self.grid.width, self.grid.height))

      # Asignamos una celda a cada uno de los elementos de la grilla
      for cell in self.grid.coord_iter():
          cell_content, x, y = cell
          for agent in cell_content:
            grid[x][y] = agent.dirty

      return grid


# Ejecución del modelo
A continuación corremos el modelo con los valores proporcionados.

In [3]:
# Definimos el tamaño del Grid, procentaje de celdas sucias y num de agentes
GRID_WIDTH = 5
GRID_HEIGHT = 5
DIRTY_CELLS = 0.5
NUM_AGENTS = 2

# Definimos el tiempo limite de ejecucion
MAX_EXEC_TIME = 3.5

# Creamos el modelo con las valores anteriores
empty_model = Modelo(NUM_AGENTS, GRID_WIDTH, GRID_HEIGHT, DIRTY_CELLS)

# Registramos el tiempo de inicio
start_time = time.time()
total_time = str(datetime.timedelta(seconds=MAX_EXEC_TIME))

# Corremos el modelo mientras que no se cumpla el tiempo limite y que todas las celdas no esten limpias
while((time.time() - start_time) < MAX_EXEC_TIME and not empty_model.clean == 0):
  empty_model.step()

# Guardamos el tiempo que le tomó correr al modelo.
exec_time =  str(datetime.timedelta(seconds=(time.time() - start_time)))


# Datos a analizar
Obtenemos la información que almacenó el colector, este nos entregará un DataFrame de pandas que contiene toda la información. Con esta información podremos calcular lo siguiente:

- Tiempo necesario hasta que todas las celdas estén limpias (o se haya llegado al tiempo máximo).
- Porcentaje de celdas limpias después del termino de la simulación.
- Número de movimientos realizados por todos los agentes.

In [4]:
# Obtenemos la información recolectada del modelo
all_grid = empty_model.datacollector.get_model_vars_dataframe()

# Obtenemos la información recolectada de los agentes
agent_vars = empty_model.datacollector.get_agent_vars_dataframe().reset_index()

# Seleccionamos los datos de movimiento de los agentes en el ultimo paso del modelo
all_moves = agent_vars[agent_vars['Step'] == agent_vars['Step'].values[-1]].dropna()['Moves']

# Seleccionamos los datos de las celdas sucias en el ultimo paso del modelo
last_step_values = agent_vars[agent_vars['Step'] == agent_vars['Step'].values[-1]].dropna()
percentage_cc = last_step_values[last_step_values['Dirty'] == 0]

print('Porcentaje de celdas limpias después del termino de la simulacion:', (len(percentage_cc) / (GRID_WIDTH * GRID_HEIGHT)) * 100, "%")
print('Movimientos realizados por los agentes: ', all_moves.sum())
print('Tiempo de necesario para que todas las celdas esten limpias: ', exec_time, '/', total_time)

Porcentaje de celdas limpias después del termino de la simulacion: 100.0 %
Movimientos realizados por los agentes:  68
Tiempo de necesario para que todas las celdas esten limpias:  0:00:00.001994 / 0:00:03.500000


# Graficación
Graficamos la información usando `matplotlib`

In [5]:
%%capture
# Obtenemos el numero de pasos del recolector de datos
steps = len(all_grid)

fig, axs = plt.subplots(figsize=(GRID_HEIGHT, GRID_WIDTH))
axs.set_xticks([])
axs.set_yticks([])
colors = ["white","black"]
cmap1 = ListedColormap(colors)
# Se grafica al valor mayor a 2 con color gris, este es nuestro agente RobotLimpiador
cmap1.set_over("grey")
patch = plt.imshow(all_grid.iloc[0][0], cmap=cmap1, vmax=2)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])


anim = animation.FuncAnimation(fig, animate, frames=steps)

# Animación
Colores de celda:
- Gris: RobotLimpiador.
- Blanco: Baldosa limpia.
- Negro: Baldosa sucia. 

In [6]:
anim